In [6]:

# STEP 1: Install MPI
# !apt-get install -y libopenmpi-dev openmpi-bin

%%writefile parallel_quicksort.cpp
#include <iostream>
#include <algorithm>
#include <cstdlib>
#include <ctime>
#include <mpi.h>
using namespace std;
// Partition function for quicksort
int partition(int arr[], int low, int high) {
    int pivot = arr[high];
    int i = (low - 1);
    for (int j = low; j <= high - 1; j++) {
        if (arr[j] <= pivot) {
            i++;
            swap(arr[i], arr[j]);
        }
    }
    swap(arr[i + 1], arr[high]);
    return (i + 1);
}

// Recursive quicksort
void quicksort(int arr[], int low, int high) {
    if (low < high) {
        int pivot = partition(arr, low, high);
        quicksort(arr, low, pivot - 1);
        quicksort(arr, pivot + 1, high);
    }
}
int main(int argc, char* argv[]) {
    int rank, size;
    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);
    int n = 100;
    int* arr = new int[n];
    int* recvbuf = new int[n];
    int sub_arr_size = n / size;
    int* sub_arr = new int[sub_arr_size];
    if (rank == 0) {
        srand(time(0));
        for (int i = 0; i < n; i++) {
            arr[i] = rand() % 100;
        }
    }
    MPI_Scatter(arr, sub_arr_size, MPI_INT, sub_arr, sub_arr_size, MPI_INT, 0, MPI_COMM_WORLD);
    double start_time = MPI_Wtime();
    quicksort(sub_arr, 0, sub_arr_size - 1);
    MPI_Gather(sub_arr, sub_arr_size, MPI_INT, recvbuf, sub_arr_size, MPI_INT, 0, MPI_COMM_WORLD);
    double end_time = MPI_Wtime();
    double parallel_execution_time = end_time - start_time;
    double sequential_execution_time = 0.0;
    if (rank == 0) {
        double seq_start = MPI_Wtime();
        quicksort(arr, 0, n - 1);
        double seq_end = MPI_Wtime();
        sequential_execution_time = seq_end - seq_start;
        cout << "Sorted Array (Parallel + Merged): ";
        for (int i = 0; i < n; i++) {
            cout << recvbuf[i] << " ";
        }
        cout << endl;
        double speedup = sequential_execution_time / parallel_execution_time;
        double efficiency = speedup / size;
        cout << "Sequential execution time: " << sequential_execution_time << endl;
        cout << "Parallel execution time: " << parallel_execution_time << endl;
        cout << "Speedup: " << speedup << endl;
        cout << "Efficiency: " << efficiency << endl;
    }
    delete[] arr;
    delete[] recvbuf;
    delete[] sub_arr;
    MPI_Finalize();
    return 0;
}

// // STEP 3: Compile the code with mpic++
// !mpic++ -o parallel_quicksort parallel_quicksort.cpp
// // STEP 4: Run the executable with mpirun
// !mpirun --allow-run-as-root --oversubscribe -np 4 ./parallel_quicksort

Overwriting parallel_quicksort.cpp


In [7]:
!apt-get install -y libopenmpi-dev openmpi-bin
!mpic++ -o parallel_quicksort parallel_quicksort.cpp

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libopenmpi-dev is already the newest version (4.1.2-2ubuntu1).
openmpi-bin is already the newest version (4.1.2-2ubuntu1).
openmpi-bin set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [8]:
!mpirun --allow-run-as-root --oversubscribe -np 4 ./parallel_quicksort

Sorted Array (Parallel + Merged): 18 20 22 26 32 39 43 56 56 60 62 69 74 74 78 80 82 85 87 90 90 93 94 96 98 0 4 6 9 12 14 17 18 20 27 41 42 44 46 49 55 56 59 60 63 69 76 94 97 99 0 3 4 14 14 14 18 22 24 25 32 33 45 51 51 54 56 64 65 69 73 76 80 81 83 2 5 13 13 20 24 28 29 35 37 39 46 56 59 63 64 65 66 67 71 71 86 87 94 95 
Sequential execution time: 1.4803e-05
Parallel execution time: 0.000214983
Speedup: 0.0688566
Efficiency: 0.0172142
